In [4]:
# 필요한 모듈 임포트
import os
from dotenv import load_dotenv
import httpx

class CustomHTTPClient(httpx.Client):
    def __init__(self, *args, **kwargs):
        kwargs.pop("proxies", None) # Remove the 'proxies' argument if present
        super().__init__(*args, **kwargs)

from openai import OpenAI

#환경변수 취득
load_dotenv()
#OpenAI API 클라이언트 생성
client = OpenAI(api_key=os.environ['API_KEY'], http_client=CustomHTTPClient())

# 모델명
model_name = "gpt-3.5-turbo-0125"

In [5]:
# 메시지 설정
message = "언어모델을 사용할 때 포인트를 알고 싶어"

# API 요청
response = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "user", "content": message}
    ],
    n = 3,
    max_tokens = 100,
    presence_penalty = 2.0
)

# print(response.choices[0].message.content.strip())
# 결과 표시
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())


APIConnectionError: Connection error.

In [31]:
# 역할을 설정
role = "당신은 일본주재 IT업계에서 일하는 회사원입니다."
# 메시지 설정
message = "추천하는 점심 메뉴를 뭔가요?"

# API 요청
response = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "system", "content": role},
        {"role": "user", "content": message}
    ],
    n = 3,
    max_tokens = 100,
    stop = "라멘"
)

# 결과 표시
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())

--------------------
일본에서 일하는 저는 일본 음식을 추천해 드리겠습니다. 가볍고 맛있는 덮밥류나 라멘이나 우동 등의 면 요리가 좋을 수 있습니다. 현지 식당에서 일본식 덮밥인 규동(규덮밥), 카츠
--------------------
일본에서 점심으로 많이 먹는 음식 중에서 추천하자면, 덮밥류나 라면이 좋습니다. 

1. 규동(소고기 덮밥): 소고기와 양파를 달달한 소스와 함께 밥 위에 올려 먹는 음식입니다.
--------------------
안녕하세요! 점심 메뉴를 추천해 드리겠습니다. 만약 일본에서 맛있는 음식을 즐기고 싶으시다면 라멘, 우동, 돈부리, 오꼬노미야끼, 회덮밥 등을 추천해드립니다. 물론


In [15]:
import tiktoken
from tiktoken.core import Encoding

encoding: Encoding = tiktoken.encoding_for_model(model_name)

In [33]:
tokens = encoding.encode("Good morning! Could you tell me where I can throw away the trash?")
tokens_count = len(tokens)
print(f"{tokens_count=}")
print(f"{tokens=}")

tokens_count=15
tokens=[15571, 6693, 0, 16910, 499, 3371, 757, 1405, 358, 649, 2571, 3201, 279, 23701, 30]


In [34]:
for token in tokens:
    bytes = encoding.decode_tokens_bytes([token])[0]
    try:
        print(bytes.decode('utf-8'), end=", ")
    except UnicodeDecodeError:
        print(bytes, end=", ")

Good,  morning, !,  Could,  you,  tell,  me,  where,  I,  can,  throw,  away,  the,  trash, ?, 

In [38]:
message = """
A씨와 B씨가 대화를 나눕니다.
A: Hello!
B:
"""
# API 요청
response = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "user", "content": message}
    ],
    n = 3,
    max_tokens = 300,
    logit_bias = {15571:8, 6693:8}    
)
# 결과 표시
for choice in response.choices:
    print("-" * 20)
    print(choice.message.content.strip())



--------------------
Hello! How are you today?
--------------------
Hello! How are you?
--------------------
B: Hi there! How are you doing today?


In [40]:
# 메시지 설정
message = "언어모델을 사용할 때 포인트를 알고 싶어"

# API 요청
stream = client.chat.completions.create(
    model = model_name,
    messages = [
        {"role": "user", "content": message}
    ],
    stream=True,
)

for chunk in stream:
    if chunk.choices:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")


언어 모델을 사용할 때 알아두어야 할 몇 가지 포인트가 있습니다:

1. 데이터의 품질: 언어 모델은 학습하는 데이터의 품질에 매우 영향을 받습니다. 따라서 가능한 적절한 데이터를 사용하여 모델을 학습시키는 것이 중요합니다.

2. 모델의 크기: 언어 모델의 성능은 일반적으로 모델의 크기와 복잡성에 영향을 받습니다. 더 큰 모델은 더 많은 데이터를 학습시킬 수 있지만 계산 및 메모리 요구 사항이 증가합니다.

3. 하이퍼파라미터 조정: 언어 모델을 사용할 때는 학습률, 배치 크기, 에포크 수 등과 같은 하이퍼파라미터를 적절히 조정해야 합니다. 이를 통해 모델의 성능을 향상시킬 수 있습니다.

4. 평가 및 튜닝: 학습된 모델을 평가하고 필요한 경우 필요한 튜닝을 수행하여 모델의 정확성과 일반적인 성능을 향상시켜야 합니다.

5. 활용: 마지막으로, 언어 모델을 사용할 때는 모델이 실제 어플리케이션에서 어떻게 활용될지 고려해야 합니다. 모델이 일반화 및 적용 가능한지 확인하고 최적의 결과를 얻을 수 있도록 적절한 조치를 취해야 합니다.

In [42]:
# 메시지를 저장하는 리스트
messages = []
while(True):
    # 사용자로부터 질문을 받음
    message = input("메시지를 입력:")
    # 질문이 입력되지 않았다면 종료
    if message.strip() =="":
        break
    display(f"질문:{message}")

    # 메시지에 사용자로부터의 질문을 추가
    messages.append({"role":"user", "content": message.strip()})

    # 메시지의 갯수가 8개를 넘으면, 오래된 메시지부터 삭제
    if len(messages) > 8:
        del_message = messages.pop()

    # API 요청
    stream = client.chat.completions.create(
        model = model_name,
        messages = messages,
        stream = True,
    )

    # 언어 모델로부터 응답을 표시
    response_message = ""
    for chunk in stream:
        if chunk.choices:
            next = chunk.choices[0].delta.content
            if next is not None:
                response_message += next
                print(next, end='')

    
    messages.append({"role": "assistant", "content": response_message})

print("\n--- 이용해 주셔서 감사합니다! ---")

'질문:안녕하세요?'

안녕하세요! 무엇을 도와드릴까요?

'질문:앙팡맨에 대해 알고 싶어'

앙팡맨은 대한민국의 인기 어린이 노래이며, 그 노래에 맞춰 춤을 추고 놀이를 하는 것을 말합니다. 앙팡맨은 원래 일본의 유명 가수, 앙パン맨에서 영향을 받았으며, 한국에서는 앙팡맨 노래와 춤이 인기를 누리고 있습니다. 많은 어린이들이 즐기는 노래이며, 아이들뿐만 아니라 부모님들도 함께 즐길 수 있는 즐거운 노래입니다. 혹시 더 궁금한 점이 있으시면 알려주세요.

'질문:일본 애니메이션 앙팡맨에 대해 알고 싶어'

앙팡맨은 일본의 만화가 타카나시 요시히코가 만든 인기 만화 캐릭터입니다. 앙팡맨은 "진짜 어린이들을 생각하는 슈퍼 히어로"로서, 어린이들을 위해 범죄와 악당을 물리치며 도와주는 이야기를 다루고 있습니다. 앙팡맨은 건강하고 맛있는 빵을 나르는 빵도둑 박스와 그의 친구들로 이루어진 이야기로, 사회적 가치와 선한 마음을 강조하는 내용으로 인기를 끌고 있습니다. 시리즈에는 다양한 캐릭터와 이야기가 있으며, 어린이들을 위한 교육적인 메시지를 전달하는 것이 특징입니다. 혹시 더 궁금한 사항이 있으시면 알려주세요.
--- 이용해 주셔서 감사합니다! ---
